# Discrete optimization: traveling salesman problem

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

from matplotlib import pyplot as plt
from networkx import (
    Graph,
    draw_networkx_nodes,
    draw_networkx_labels,
    draw_networkx_edges,
    draw_networkx_edge_labels,
)


This exercise does not require to code in Python.

Anna and Tom live in London and want to plan a road trip during
their holidays. They want to visit 3 cities: Barcelona, Budapest and
Prague. They would like to minimize the cost of their trip by
ordering the cities in a smart way. The cost to go

- between Barcelona and Budapest is 150.- CHF,
- between Barcelona and Prague is 170.- CHF,
- between Barcelona and London is 90.- CHF,
- between London and Prague is 120.- CHF,
- between London and Budapest is 200.- CHF,
- between Budapest and Prague is 70.- CHF,

as summarized in the figure below.

Building the graph

In [ ]:
G = Graph()
positions = {
    'Barcelona': (-5, 2),
    'Budapest': (-5, -0.5),
    'London': (0, 2),
    'Prague': (0, -0.5),
}
G.add_node('Barcelona')
G.add_node('Budapest')
G.add_node('London')
G.add_node('Prague')


Arcs with distances

In [ ]:
arcs = [
    ("Barcelona", "Budapest", 150),
    ("Barcelona", "Prague", 170),
    ("Barcelona", "London", 90),
    ("Budapest", "Prague", 70),
    ("Budapest", "London", 200),
    ("London", "Prague", 120),
]
for arc in arcs:
    G.add_edge(arc[0], arc[1], weight=arc[2])
node_colors = ['lightgray' if node == 'London' else 'white' for node in G.nodes()]
edge_labels = {(edge[0], edge[1]): edge[2] for edge in arcs}



Show the plot

In [ ]:
draw_networkx_nodes(
    G,
    pos=positions,
    node_color=node_colors,
    edgecolors='black',
    node_size=2000,
    alpha=0.5,
)
draw_networkx_labels(G, pos=positions, font_size=12)
draw_networkx_edges(G, pos=positions, edge_color='black', arrows=False)
draw_networkx_edge_labels(
    G, pos=positions, edge_labels=edge_labels, font_size=10, label_pos=0.3
)
plt.axis('off')
plt.show()


# Question 1
Model that traveling salesman problem as an integer optimization problem.

We denote $\mathcal{N}$ the set of $n$ nodes. The decision variables are binary
variables $x_{ij}$, with $i \in \mathcal{N}$, $j \in \mathcal{N}$ that are equal
to $1$ if Anna and Tom visit city $j$ just after city $i$, and is equal to
$0$ otherwise. As Anna and Tom want to minimize their travel costs,
the objective function is:

\begin{align*}
\min_{x \in \{0,1\}^{12}} & 150 x_{B,Bu} + 170 x_{B,P} + 90 x_{B,L}\\
& +120 x_{L,P} + 200 x_{L,Bu} + 90 x_{L,B} \\
& +70 x_{Bu,P} + 150 x_{Bu,B} + 200 x_{Bu,L} \\
& +170 x_{P,B} + 120 x_{P,L} + 70 x_{P,Bu}.
\end{align*}

Each city must have exactly one successor, that is
\begin{align*}
x_{B,Bu} + x_{B,P} + x_{B,L} & = 1,\\
x_{L,P} + x_{L,Bu} + x_{L,B} & = 1,\\
x_{Bu,P} + x_{Bu,B} + x_{Bu,L} & = 1,\\
x_{P,B} + x_{P,L} + x_{P,Bu} & =1.
\end{align*}

Furthermore, each city must have exactly one predecessor, that is
\begin{align*}
x_{Bu,B} + x_{P,B} + x_{L,B} & = 1,\\
x_{P,L} + x_{Bu,L} + x_{B,L} & = 1,\\
x_{P,Bu} + x_{B,Bu} + x_{L,Bu} & = 1,\\
x_{B,P} + x_{L,P} + x_{Bu,P} & = 1.
\end{align*}

Finally, to eliminate subtours, we introduce the integer variables
$y_i$, for each city $i$ except London, modeling the position of city
$i$ in the tour.  The subtour elimination constraint imposes that, if
$x_{ij} = 1$, then $y_j \geq y_i + 1$. It can be modeled as
$$
x_{ij}(n-1) \leq y_j - y_i + n - 2, \qquad \forall i,j \in \mathcal{N} \setminus
\{\text{London}\},
$$
where $n=4$.

If $x_{ij} = 1$, that is, if city $j$ is visited just after city $i$,
the constraint becomes
$$
1 \cdot 3 \leq y_j - y_i + 4 - 2,
$$
or, equivalently,
$$
y_j  \geq y_i+ 1,
$$
that is, the position of city $j$ in the tour must be after city $j$.


If $x_{ij} = 0$, that is if city $j$ is not the city visited after
city $i$, the constraint becomes
$$
0 \leq y_j - y_i + 4 - 2,
$$
or, equivalently,
$$
y_i - y_j \leq 2,
$$
which is always verified. Indeed, as London is not numbered, the numbering
can be 1, 2, or 3, and the largest difference is  2. In general, any
difference $y_j - y_i$ is less or equal to $n-2$.

The subtour elimination constraints are:
\begin{align*}
3x_{B,Bu} + y_{B} - y_{Bu} & \leq 2,\\
3x_{Bu,B} + y_{Bu} - y_{B} & \leq 2,\\
3x_{P,B} + y_{P} - y_{B} & \leq 2,\\
3x_{B,P} + y_{B} - y_{P} & \leq 2,\\
3x_{Bu,P} + y_{Bu} - y_{P} & \leq 2,\\
3x_{P,Bu} + y_{P} - y_{Bu} & \leq 2.
\end{align*}

To sum up, the mathematical model for the given problem is the following:

\begin{align*}
\min_{x \in \{0,1\}^{12}}  & 150 x_{B,Bu} + 170 x_{B,P} + 90 x_{B,L} & \\
& + 120 x_{L,P} + 200 x_{L,Bu} + 90 x_{L,B} & \\
& + 70 x_{Bu,P} + 150 x_{Bu,B} + 200 x_{Bu,L} & \\
& + 170 x_{P,B} + 120 x_{P,L} + 70 x_{P,Bu} & \\
\text{subject to} \quad& & \\
&x_{B,Bu} + x_{B,P} + x_{B,L} = 1, &\\
&x_{L,P} + x_{L,Bu} + x_{L,B} = 1, &\\
&x_{Bu,P} + x_{Bu,B} + x_{Bu,L} = 1, &\\
&x_{P,B} + x_{P,L} + x_{P,Bu} = 1, &\\
&x_{Bu,B} + x_{P,B} + x_{L,B} = 1, &\\
&x_{P,L} + x_{Bu,L} + x_{B,L} = 1, &\\
&x_{P,Bu} + x_{B,Bu} + x_{L,Bu} = 1, &\\
&x_{B,P} + x_{L,P} + x_{Bu,P} = 1, &\\
&3x_{B,Bu} + y_{B} - y_{Bu} \leq 2, &\\
&3x_{Bu,B} + y_{Bu} - y_{B} \leq 2, &\\
&3x_{P,B} + y_{P} - y_{B} \leq 2, &\\
&3x_{B,P} + y_{B} - y_{P} \leq 2, &\\
&3x_{Bu,P} + y_{Bu} - y_{P} \leq 2, &\\
&3x_{P,Bu} + y_{P} - y_{Bu} \leq 2, &\\
&y_{i} \in \{1,2,3\}, \qquad \forall i \in \mathcal{N} \setminus \{L\}, &\\
&x_{i,j} \in \{0,1\}, \qquad \forall i,j \in \mathcal{N}. &\\
\end{align*}

# Question 2
Consider one possible tour. Provide the corresponding values of
the decision variables of the optimization problem, and show that they verify
the constraints.

Consider the tour London $\to$ Barcelona $\to$ Budapest $\to$
Prague. The corresponding decision variables are
\begin{align*}
x_{L,B} &= 1, & x_{L,Bu} &= 0, & x_{L,P} &= 0, \\
x_{B,L} &= 0, & x_{B,Bu} &= 1, & x_{B,P} &= 0, \\
x_{Bu,L} &= 0, & x_{Bu, B} &= 0, & x_{Bu,P} &= 1, \\
x_{P,L} &= 1, & x_{P,B} &= 0, & x_{P,Bu} &= 0, \\
y_{B} &= 1, & y_{Bu} &= 2, & y_{P} &= 3.
\end{align*}

We can verify the constraints:

\begin{align*}
x_{B,Bu} + x_{B,P} + x_{B,L} &= 1 + 0 + 0 &= 1, \\
x_{L,P} + x_{L,Bu} + x_{L,B} &= 0 + 0 + 1 &= 1, \\
x_{Bu,P} + x_{Bu,B} + x_{Bu,L} &= 1 + 0 + 0 &= 1, \\
x_{P,B} + x_{P,L} + x_{P,Bu} &= 0 + 1 + 0 &= 1, \\
x_{Bu,B} + x_{P,B} + x_{L,B} &= 0 + 0 + 1 &= 1, \\
x_{P,L} + x_{Bu,L} + x_{B,L} &= 1 + 0 + 0 &= 1, \\
x_{P,Bu} + x_{B,Bu} + x_{L,Bu} &= 0 + 1 + 0 &= 1, \\
x_{B,P} + x_{L,P} + x_{Bu,P} &= 0 + 0 + 1 &= 1, \\
3x_{B,Bu} + y_{B} - y_{Bu} &= 3 + 1 - 2 = 2&\leq 2, \\
3x_{Bu,B} + y_{Bu} - y_{B} &= 2-1 = 1&\leq 2, \\
3x_{P,B} + y_{P} - y_{B} &= 3-1=2&\leq 2, \\
3x_{B,P} + y_{B} - y_{P} &= 1 - 3 = -2 &\leq 2, \\
3x_{Bu,P} + y_{Bu} - y_{P} &= 3+2-3=2&\leq 2, \\
3x_{P,Bu} + y_{P} - y_{Bu} &= 3-2=1 &\leq 2. \\
\end{align*}